In [1]:
import torch
import utilities
import numpy as np
import pandas as pd
from sc_sharp import scSHARP

In [2]:
x = np.array([[1,1], [2,2], [3,3], [10,10], [11,11], [12,12]])
preds = [0, -1, 0, 1, -1, 1]

In [3]:
utilities.knn_consensus(x, preds, 2)

array([ 0, -1,  0,  1, -1,  1])

In [4]:
counts = pd.read_csv("simulations/splat_0.7_de_rq/query_counts.csv", index_col = 0)
counts.head()

,Gene1,Gene2,Gene3,Gene4,Gene5,Gene6,Gene7,Gene8,Gene9,Gene10,...,Gene33685,Gene33686,Gene33687,Gene33688,Gene33689,Gene33690,Gene33691,Gene33692,Gene33693,Gene33694
Cell1001,0,0,0,0,1,6,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
Cell1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cell1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cell1004,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
Cell1005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
all_labels = pd.read_csv("simulations/splat_0.7_de_rq/preds.csv", index_col=0)
all_labels.head()

,scina,scsorter,sctype,singler,scpred
Cell1001,Group2,Group2,Group2,Group2,Group2
Cell1002,Group4,Group2,Group2,Group4,Group2
Cell1003,Group1,Group4,Group2,Group2,NaN
Cell1004,Group4,Group4,Group4,Group4,Group1
Cell1005,Group1,Group1,Group1,Group1,Group1


In [6]:
X, keep_cells,keep_genes,pca_obj = utilities.preprocess(np.array(counts), scale=False, run_pca=False)
X.shape

(999, 16196)

In [7]:
_,marker_names = utilities.read_marker_file("simulations/splat_0.7_de_rq/markers.txt")

In [8]:
all_labels_factored = utilities.factorize_df(all_labels, marker_names)
encoded_labels = utilities.encode_predictions(all_labels_factored)
confident_labels = utilities.get_consensus_labels(encoded_labels, necessary_vote = .51)
confident_labels = confident_labels[keep_cells]

In [9]:
real_y = pd.read_csv("simulations/splat_0.7_de_rq/query_meta.csv", index_col= 0).Group
real_y = pd.factorize(real_y, sort=True)[0]
real_y = real_y[keep_cells]

In [11]:
for i in [10,50,100, 200, 400]:
    preds = utilities.knn_consensus(X, confident_labels, i)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])


1
10
(0.8068068027496338, array([[  0,   0,   0,   0,   0],
       [  0, 232,   0,   0,   0],
       [  0,  25, 239,   4,   0],
       [  5,  60,   3, 163,   3],
       [  4,  76,   5,   8, 172]]))
1
50
(0.9029029011726379, array([[  0,   0,   0,   0,   0],
       [  0, 232,   0,   0,   0],
       [  0,   5, 262,   1,   0],
       [  3,  44,   3, 183,   1],
       [  0,  39,   0,   1, 225]]))
1
100
(0.9339339137077332, array([[  0,   0,   0,   0,   0],
       [  0, 231,   1,   0,   0],
       [  0,   1, 266,   1,   0],
       [  2,  30,   3, 196,   3],
       [  0,  21,   3,   1, 240]]))
1
200
(0.9639639854431152, array([[  0,   0,   0,   0,   0],
       [  0, 229,   2,   0,   1],
       [  0,   0, 268,   0,   0],
       [  1,  10,   7, 213,   3],
       [  1,   9,   0,   2, 253]]))
